## Análisis de indicadores INEGI de educación y población: estatal y nacional

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests # Acceder a páginas web
import json
import math

#### Indicadores seleccionados

In [2]:
dic_indicadores = {
    1:[1002000041,'Porcentaje de personas de 15 años y más alfabetas'],
    2:[1005000038,'Grado promedio de escolaridad de la población de 15 y más años'],
    3:[3108001002,'Porcentaje de analfabetas hombres'],
    4:[3108001003 ,'Porcentaje de analfabetas mujeres'],
    5:[6200027788,'Eficiencia terminal en educación media superior'],
    6:[6200205239,'Población de 15 años y más con algún grado aprobado en educación básica'],
    7:[6200205240,'Población de 19 años y más con algún grado aprobado en estudios técnicos o comerciales con secundaria terminada'],
    8:[6200205241,'Población de 19 años y más con algún grado aprobado en bachillerato'],
    9:[6200205242,'Población de 24 años y más con algún grado aprobado en estudios superiores'],
    10:[6207067825,'Gasto nacional en educación total como porcentaje del PIB'], #Únicamente válido para totales
    11:[1002000001,'Población total'],
    12:[1002000002,'Población total hombres'],
    13:[1002000003,'Población total mujeres']
}

### 1.  Análisis nacional

#### 1.1. URLs

In [3]:
# Token enviado por correo
token = 'token'

# Url para indicadores nacionales sobre nivel de educación y analfabetismo
url_educacionNacional = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000041,1005000038,3108001002,3108001003,6200205239,6200205240,6200205241,6200205242,6200027788,6207067825/es/0700/false/BISE/2.0/{token}?type=json'

# Url para población total, hombres y mujeres nacional
url_poblacionNacional = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000001,1002000002,1002000003/es/0700/false/BISE/2.0/{token}?type=json'

# Lista con ambos urls
lista_conjuntoNacional = [url_educacionNacional, url_poblacionNacional]

# Donde se almacenará la información de los urls
contenidoNacional = []

#### 1.2. Guardado de contenido

In [4]:
for i in range(2):
    respuesta = requests.get(lista_conjuntoNacional[i])
    if (respuesta.status_code == 200):
        contenidoNacional.append(json.loads(respuesta.content))

contenidoNacional

[{'Header': {'Name': 'Datos compactos BISE',
   'Email': 'atencion.usuarios@inegi.org.mx'},
  'Series': [{'INDICADOR': '1002000041',
    'FREQ': '1',
    'TOPIC': '15',
    'UNIT': '3',
    'UNIT_MULT': '',
    'NOTE': '',
    'SOURCE': '2,487,1714,3001',
    'LASTUPDATE': '25/05/2022',
    'STATUS': '3',
    'OBSERVATIONS': [{'TIME_PERIOD': '2000',
      'OBS_VALUE': '90.45080666409960000000',
      'OBS_EXCEPTION': None,
      'OBS_STATUS': '3',
      'OBS_SOURCE': '',
      'OBS_NOTE': '9',
      'COBER_GEO': '0700'},
     {'TIME_PERIOD': '2010',
      'OBS_VALUE': '92.35000000000000000000',
      'OBS_EXCEPTION': None,
      'OBS_STATUS': '3',
      'OBS_SOURCE': '',
      'OBS_NOTE': '115',
      'COBER_GEO': '0700'},
     {'TIME_PERIOD': '2015',
      'OBS_VALUE': '93.62347048918600000000',
      'OBS_EXCEPTION': None,
      'OBS_STATUS': '3',
      'OBS_SOURCE': '',
      'OBS_NOTE': '8469',
      'COBER_GEO': '0700'},
     {'TIME_PERIOD': '2020',
      'OBS_VALUE': '95.03685329

#### 1.3. Tamaño de indicadores sobre educación y población

In [5]:
# Número de indicadores de grados de estudio y población

num_indicadores_educacionNacional = len(contenidoNacional[0]['Series'])
num_indicadores_poblacionNacional = len(contenidoNacional[1]['Series'])

print(f'Total de indicadores de educación nacionales: {num_indicadores_educacionNacional}')
print(f'Total de indicadores de población nacionales: {num_indicadores_poblacionNacional}')

Total de indicadores de educación nacionales: 10
Total de indicadores de población nacionales: 3


#### 1.4. Adqusición de periodo de tiempo y valor observado para indicadores de educación

In [6]:
obs_porIndicador_educacion = {}

for i in range(num_indicadores_educacionNacional):
    # Agarra el valor (string en este caso) de la llave "INDICADOR"
    indicador_educacion = contenidoNacional[0]['Series'][i]['INDICADOR']
    
    # Agarra el valor (lista con diccionarios) de la llave "OBSERVATIONS" 
    obs_educacion = contenidoNacional[0]['Series'][i]['OBSERVATIONS'] 
    
    # Valor (int) cambiante, según la cantidad de diccionarios (dividida por periodo de tiempo/años) para cada llave "OBSERVATIONS"
    num_obs_educacion = len(obs_educacion) 

    # Añade dos llaves por cada indicador: 
    #   'Year' para las listas de "TIME_PERIOD",
    #   'Value' para las listas de "OBS_VALUE"
    observations_educacion = {
        'Year' : [int(obs_educacion[j]['TIME_PERIOD']) for j in range(num_obs_educacion)],
        'Value' : [float(obs_educacion[j]['OBS_VALUE'] or -1) for j in range(num_obs_educacion)]
    }

    # Añade llave con valor int correspondiente al número de indicador especificado en "dic_indicadores"
    obs_porIndicador_educacion[i + 1] = observations_educacion

obs_porIndicador_educacion

{1: {'Year': [2000, 2010, 2015, 2020],
  'Value': [90.4508066640996, 92.35, 93.623470489186, 95.0368532952485]},
 2: {'Year': [1995, 2000, 2005, 2010, 2015, 2020],
  'Value': [-1.0, 7.5, 8.1, 8.63, 9.16184342150625, 9.73805046766345]},
 3: {'Year': [1970, 1990, 1995, 2000, 2005, 2010, 2015],
  'Value': [21.82, 9.63, 8.42, 7.43, 6.8, 5.57, 4.41453140179298]},
 4: {'Year': [1970, 1990, 1995, 2000, 2005, 2010, 2015],
  'Value': [29.63, 15.01, 12.66, 11.31, 9.77, 8.08, 6.45082052388037]},
 5: {'Year': [2000,
   2001,
   2002,
   2003,
   2004,
   2005,
   2006,
   2007,
   2008,
   2009,
   2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2019],
  'Value': [57.0192622288855,
   57.1848362476372,
   59.2584716168856,
   58.4012938553559,
   58.01510386733,
   58.2515864138516,
   58.0173617282181,
   58.9411454899431,
   60.8514379458176,
   61.9903518836648,
   62.234192458259,
   61.2920155088929,
   65.5841985224562,
   63.2439249406801,
   68.0765909365605,
   64.818447746

#### 1.4.1.   DataFrame para indicadores de educación

In [16]:
lista_educacionNacional = []

for i in range(num_indicadores_educacionNacional):
    lista_educacionNacional.append(pd.DataFrame(obs_porIndicador_educacion[i + 1]).set_index('Year').rename(
        columns = {'Value': f'Value (ind: {i + 1})'}))
    
    df_educacionNacional = pd.concat(lista_educacionNacional, axis = 1)

df_educacionNacional.sort_index(inplace = True)
df_educacionNacional

,Value (ind: 1),Value (ind: 2),Value (ind: 3),Value (ind: 4),Value (ind: 5),Value (ind: 6),Value (ind: 7),Value (ind: 8),Value (ind: 9),Value (ind: 10)
Year,,,,,,,,,,
1970,NaN,NaN,21.820000,29.630000,NaN,NaN,NaN,NaN,NaN,NaN
1990,NaN,NaN,9.630000,15.010000,NaN,NaN,1933234.0,3131703.0,3088416.0,NaN
1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.341431
1995,NaN,-1.000000,8.420000,12.660000,NaN,NaN,NaN,NaN,NaN,3.898544
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.760656
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.748410
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.126106
1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.231578
2000,90.450807,7.500000,7.430000,11.310000,57.019262,38761123.0,2445928.0,5582461.0,5486099.0,5.274412


#### 1.5. Adquisición de periodo de tiempo y valor observado para indicadores de población

In [8]:
obs_porIndicador_poblacion = {}

for i in range(num_indicadores_poblacionNacional):
    # Agarra el valor (string en este caso) de la llave "INDICADOR"
    indicador_poblacion = contenidoNacional[1]['Series'][i]['INDICADOR']
    
    # Agarra el valor (lista con diccionarios) de la llave "OBSERVATIONS" 
    obs_poblacion = contenidoNacional[1]['Series'][i]['OBSERVATIONS'] 
    
    # Valor (int) cambiante, según la cantidad de diccionarios (dividida por periodo de tiempo/años) para cada llave "OBSERVATIONS"
    num_obs_poblacion = len(obs_poblacion) 

    # Añade dos llaves por cada indicador: 
    #   'Year' para las listas de "TIME_PERIOD",
    #   'Value' para las listas de "OBS_VALUE"
    observations_poblacion = {
        'Year' : [int(obs_poblacion[j]['TIME_PERIOD']) for j in range(num_obs_poblacion)],
        'Value' : [float(obs_poblacion[j]['OBS_VALUE'] or -1) for j in range(num_obs_poblacion)]
    }

    # Añade llave con valor int correspondiente al número de indicador especificado en "dic_indicadores"
    obs_porIndicador_poblacion[i + 1] = observations_poblacion

obs_porIndicador_poblacion

{1: {'Year': [1910,
   1921,
   1930,
   1940,
   1950,
   1960,
   1970,
   1980,
   1990,
   1995,
   2000,
   2005,
   2010,
   2015,
   2020],
  'Value': [15160369.0,
   14334780.0,
   16552722.0,
   19653552.0,
   25791017.0,
   34923129.0,
   48225238.0,
   66846833.0,
   81249645.0,
   91158290.0,
   97483412.0,
   103263388.0,
   112336538.0,
   119938473.0,
   126014024.0]},
 2: {'Year': [1910,
   1921,
   1930,
   1940,
   1950,
   1960,
   1970,
   1980,
   1990,
   1995,
   2000,
   2005,
   2010,
   2020],
  'Value': [7504471.0,
   7003785.0,
   8119004.0,
   9695787.0,
   12696935.0,
   17415320.0,
   24065614.0,
   33039307.0,
   39893969.0,
   44900499.0,
   47592253.0,
   50249955.0,
   54855231.0,
   61473390.0]},
 3: {'Year': [1910,
   1921,
   1930,
   1940,
   1950,
   1960,
   1970,
   1980,
   1990,
   1995,
   2000,
   2005,
   2010,
   2020],
  'Value': [7655898.0,
   7330995.0,
   8433718.0,
   9957765.0,
   13094082.0,
   17507809.0,
   24159624.0,
   3380752

#### 1.5.1.    DataFrame para indicadores de población

In [9]:
lista_poblacionNacional = []

for i in range(num_indicadores_poblacionNacional):
    lista_poblacionNacional.append(pd.DataFrame(obs_porIndicador_poblacion[i + 1]).set_index('Year').rename(
        columns = {'Value': f'Value (ind: {i + 11})'}))
    
    df_poblacionNacional = pd.concat(lista_poblacionNacional, axis = 1)

df_poblacionNacional.sort_index(inplace = True)
df_poblacionNacional

,Value (ind: 11),Value (ind: 12),Value (ind: 13)
Year,,,
1910,15160369.0,7504471.0,7655898.0
1921,14334780.0,7003785.0,7330995.0
1930,16552722.0,8119004.0,8433718.0
1940,19653552.0,9695787.0,9957765.0
1950,25791017.0,12696935.0,13094082.0
1960,34923129.0,17415320.0,17507809.0
1970,48225238.0,24065614.0,24159624.0
1980,66846833.0,33039307.0,33807526.0
1990,81249645.0,39893969.0,41355676.0


### 2.  Análisis por estados

#### 2.1.    Guardado de contenido

In [10]:
token = 'token'

contenidoEstados = []

for i in range(64):
    
    if (i <= 31): # Indicadores de educación
        url_estados = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000041,1005000038,3108001002,3108001003,6200205239,6200205240,6200205241,6200205242,6200027788/es/0{str(7000001+i)}/false/BISE/2.0/{token}?type=json'
        
    else: # Indicadores de población
        url_estados = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000001,1002000002,1002000003/es/0{str(7000001+i-32)}/false/BISE/2.0/{token}?type=json'
        
    respuesta = requests.get(url_estados)
    
    if respuesta.status_code == 200:
        contenidoEstados.append(json.loads(respuesta.content))

contenidoEstados

[{'Header': {'Name': 'Datos compactos BISE',
   'Email': 'atencion.usuarios@inegi.org.mx'},
  'Series': [{'INDICADOR': '1002000041',
    'FREQ': '1',
    'TOPIC': '15',
    'UNIT': '3',
    'UNIT_MULT': '',
    'NOTE': '',
    'SOURCE': '2,487,1714,3001',
    'LASTUPDATE': '25/05/2022',
    'STATUS': '3',
    'OBSERVATIONS': [{'TIME_PERIOD': '2000',
      'OBS_VALUE': '95.10677635655600000000',
      'OBS_EXCEPTION': None,
      'OBS_STATUS': '3',
      'OBS_SOURCE': '',
      'OBS_NOTE': '9',
      'COBER_GEO': '07000001'},
     {'TIME_PERIOD': '2010',
      'OBS_VALUE': '96.20000000000000000000',
      'OBS_EXCEPTION': None,
      'OBS_STATUS': '3',
      'OBS_SOURCE': '',
      'OBS_NOTE': '115',
      'COBER_GEO': '07000001'},
     {'TIME_PERIOD': '2015',
      'OBS_VALUE': '97.03229024353220000000',
      'OBS_EXCEPTION': None,
      'OBS_STATUS': '3',
      'OBS_SOURCE': '',
      'OBS_NOTE': '8469',
      'COBER_GEO': '07000001'},
     {'TIME_PERIOD': '2020',
      'OBS_VALUE': 

#### 2.2.   Tamaño de indicadores sobre educación y población

In [11]:
num_estados = 32 

num_indicadores_poblacionEstados = len(contenidoEstados[32]['Series'])
num_indicadores_educacionEstados = len(contenidoEstados[0]['Series']) 

print(f'Total de indicadores de población para todos los estados: {num_indicadores_poblacionEstados}')
print(f'Total de indicadores de educación para todos los estados: {num_indicadores_educacionEstados}')

Total de indicadores de población para todos los estados: 3
Total de indicadores de educación para todos los estados: 9


#### 2.3.   Adquisición de periodo de tiempo y valor observado para indicadores de educación

In [12]:
df_estados = pd.read_csv('datos/Catalogo_entidades.csv')[:32]

obs_porEstado_educacion = {}

for j in range(num_estados):
    obs_porIndicador_educacionEstados = {}
    
    for i in range(num_indicadores_educacionEstados):
        indicador_educacionEstados = contenidoEstados[j]['Series'][i]['INDICADOR']
        obs_educacionEstados = contenidoEstados[j]['Series'][i]['OBSERVATIONS']

        num_obs_educacionEstados = len(obs_educacionEstados)
        
        observations_educacionEstados = {
            'Año' : [int(obs_educacionEstados[k]['TIME_PERIOD']) for k in range(num_obs_educacionEstados)],
            'Valor' : [float(obs_educacionEstados[k]['OBS_VALUE'] or -1) for k in range(num_obs_educacionEstados)]
        }

        obs_porIndicador_educacionEstados[i+1] = observations_educacionEstados
        
    obs_porEstado_educacion[df_estados['CLAVE_ENTIDAD'][j]] = obs_porIndicador_educacionEstados

obs_porEstado_educacion

{1: {1: {'Año': [2000, 2010, 2015, 2020],
   'Valor': [95.106776356556, 96.2, 97.0322902435322, 97.7402146877864]},
  2: {'Año': [1995, 2000, 2005, 2010, 2015, 2020],
   'Valor': [-1.0, 8.0, 8.7, 9.23, 9.72597472304436, 10.3491343202918]},
  3: {'Año': [1910,
    1921,
    1930,
    1940,
    1950,
    1960,
    1970,
    1980,
    1990,
    1995,
    2000,
    2005,
    2010,
    2015],
   'Valor': [61.8,
    51.8,
    50.8,
    35.9,
    29.0,
    19.3,
    13.9,
    9.1,
    5.9,
    4.9,
    4.3,
    3.8,
    2.96,
    2.35865786245642]},
  4: {'Año': [1910,
    1921,
    1930,
    1940,
    1950,
    1960,
    1970,
    1980,
    1990,
    1995,
    2000,
    2005,
    2010,
    2015],
   'Valor': [66.6,
    55.5,
    60.8,
    43.4,
    32.8,
    25.4,
    19.2,
    12.5,
    8.1,
    6.3,
    5.4,
    4.5,
    3.52,
    2.80654638412153]},
  5: {'Año': [2000,
    2001,
    2002,
    2003,
    2004,
    2005,
    2006,
    2007,
    2008,
    2009,
    2010,
    2011,
    2012,
 

#### 2.3.1. DataFrame para indicadores de educación

In [13]:
indicador_minimo = 1

lista_educacionEstados = []

for j in range(num_estados):
    lista_porIndicador_educacionEstados = []
    
    for i in range(num_indicadores_educacionEstados):
        lista_porIndicador_educacionEstados.append(pd.DataFrame(obs_porEstado_educacion[j+1][indicador_minimo + i]).set_index('Año').rename(
            columns={'Valor':f'Valor (ind: {str(indicador_minimo+i)})'}))

    df_porIndicador_educacionEstados = pd.concat(lista_porIndicador_educacionEstados, axis = 1).sort_index()
    df_porIndicador_educacionEstados.insert(loc = 0, column = 'Entidad', value = j+1)
    lista_educacionEstados.append(df_porIndicador_educacionEstados)

df_educacionEstados = pd.concat(lista_educacionEstados, axis = 0).sort_index().reset_index().sort_values(['Año','Entidad']).reset_index(drop = True)
df_educacionEstados

,Año,Entidad,Valor (ind: 1),Valor (ind: 2),Valor (ind: 3),Valor (ind: 4),Valor (ind: 5),Valor (ind: 6),Valor (ind: 7),Valor (ind: 8),Valor (ind: 9)
0,1910,1,NaN,NaN,61.8,66.6,NaN,NaN,NaN,NaN,NaN
1,1910,2,NaN,NaN,46.6,46.3,NaN,NaN,NaN,NaN,NaN
2,1910,3,NaN,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN
3,1910,4,NaN,NaN,63.9,68.9,NaN,NaN,NaN,NaN,NaN
4,1910,5,NaN,NaN,52.6,59.4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
923,2020,28,97.189445,10.092209,NaN,NaN,NaN,NaN,NaN,NaN,NaN
924,2020,29,96.507161,9.833170,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,2020,30,91.411294,8.745629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926,2020,31,93.842523,9.594718,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2.4.   Adquisición de periodo de tiempo y valor observado para indicadores de población

In [14]:
obs_porEstado_poblacion = {}

for j in range(num_estados):
    obs_porIndicador_poblacionEstados = {}
    
    for i in range(num_indicadores_poblacionEstados):
        indicador_poblacionEstados = contenidoEstados[j+32]['Series'][i]['INDICADOR']
        obs_poblacionEstados = contenidoEstados[j+32]['Series'][i]['OBSERVATIONS']

        num_obs_poblacionEstados = len(obs_poblacionEstados)
        
        observations_poblacionEstados = {
            'Año' : [int(obs_poblacionEstados[k]['TIME_PERIOD']) for k in range(num_obs_poblacionEstados)],
            'Valor' : [float(obs_poblacionEstados[k]['OBS_VALUE'] or -1) for k in range(num_obs_poblacionEstados)]
        }

        obs_porIndicador_poblacionEstados[i+1] = observations_poblacionEstados
        
    obs_porEstado_poblacion[df_estados['CLAVE_ENTIDAD'][j]] = obs_porIndicador_poblacionEstados

obs_porEstado_poblacion

{1: {1: {'Año': [1910,
    1921,
    1930,
    1940,
    1950,
    1960,
    1970,
    1980,
    1990,
    1995,
    2000,
    2005,
    2010,
    2015,
    2020],
   'Valor': [120511.0,
    107581.0,
    132900.0,
    161693.0,
    188075.0,
    243363.0,
    338142.0,
    519439.0,
    719659.0,
    862720.0,
    944285.0,
    1065416.0,
    1184996.0,
    1316032.0,
    1425607.0]},
  2: {'Año': [1995, 2000, 2005, 2010, 2020],
   'Valor': [422324.0, 456533.0, 515364.0, 576638.0, 696683.0]},
  3: {'Año': [1995, 2000, 2005, 2010, 2020],
   'Valor': [440396.0, 487752.0, 550052.0, 608358.0, 728924.0]}},
 2: {1: {'Año': [1910,
    1921,
    1930,
    1940,
    1950,
    1960,
    1970,
    1980,
    1990,
    1995,
    2000,
    2005,
    2010,
    2015,
    2020],
   'Valor': [52272.0,
    23537.0,
    48327.0,
    78907.0,
    226965.0,
    520165.0,
    870421.0,
    1177886.0,
    1660855.0,
    2112140.0,
    2487367.0,
    2844469.0,
    3155070.0,
    3348898.0,
    3769020.0]},
 

#### 2.4.1  DataFrame para indicadores de población

In [15]:
lista_poblacionEstados = []

for j in range(num_estados):
    lista_porIndicador_poblacionEstados = []
    
    for i in range(num_indicadores_poblacionEstados):
        lista_porIndicador_poblacionEstados.append(pd.DataFrame(obs_porEstado_poblacion[j+1][indicador_minimo + i]).set_index('Año').rename(
            columns={'Valor':'Valor (ind:'+str(indicador_minimo+i)+')'}))

    df_porIndicador_poblacionEstados = pd.concat(lista_porIndicador_poblacionEstados, axis = 1).sort_index()
    df_porIndicador_poblacionEstados.insert(loc = 0, column = 'Entidad', value = j+1)
    lista_poblacionEstados.append(df_porIndicador_poblacionEstados)

df_poblacionEstados = pd.concat(lista_poblacionEstados, axis = 0).sort_index().reset_index().sort_values(['Año','Entidad']).reset_index(drop=True)
df_poblacionEstados

,Año,Entidad,Valor (ind:1),Valor (ind:2),Valor (ind:3)
0,1910,1,120511.0,NaN,NaN
1,1910,2,52272.0,NaN,NaN
2,1910,3,-1.0,NaN,NaN
3,1910,4,86661.0,NaN,NaN
4,1910,5,362092.0,NaN,NaN
...,...,...,...,...,...
475,2020,28,3527735.0,1736140.0,1791595.0
476,2020,29,1342977.0,649894.0,693083.0
477,2020,30,8062579.0,3871774.0,4190805.0
478,2020,31,2320898.0,1140279.0,1180619.0
